In [1]:
from make_benchmarks import *
import os

In [ ]:
work_dir = "/data/rashika/CAFA4/"


t0_mapped_path = work_dir + "mapped/t0.csv"
t1_mapped_path = work_dir + "mapped/t1.csv"

#t0_mapped_path = work_dir + "mapped/t1_test.csv"
#t1_mapped_path = work_dir + "mapped/t0_test.csv"

t0_ont_file = '/data/rashika/CAFA4/uniprot/go_2019_12_09/go-basic.obo' 
t0_ont_graph = clean_ontology_edges(obonet.read_obo(t0_ont_file))     

t1_ont_file = "/data/rashika/CAFA4/uniprot/go_2024_03_28/go-basic.obo"
t1_ont_graph = clean_ontology_edges(obonet.read_obo(t1_ont_file)) 

t_minus_ont_file =  "/data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo"
t_minus_1_ont_graph = clean_ontology_edges(obonet.read_obo(t_minus_ont_file))

c3_t_minus_ont_file =  "/data/rashika/CAFA4/head2head/CAFA3/go.obo"
c3_t_minus_1_ont_graph = clean_ontology_edges(obonet.read_obo(c3_t_minus_ont_file))

c2_t_minus_ont_file =  "/data/rashika/CAFA4/head2head/CAFA2/go.obo"
c2_t_minus_1_ont_graph = clean_ontology_edges(obonet.read_obo(c2_t_minus_ont_file))


In [ ]:
#Concat all three ontologies of each evaluation type together

# Commands run in /data/rashika/CAFA4/head2head/eval directory

#cat BM_GO/bpo_all_type1.txt BM_GO/cco_all_type1.txt BM_GO/mfo_all_type1.txt > BM_GO_all_ont/type1.txt 
#cat BM_GO/bpo_all_type2.txt BM_GO/cco_all_type2.txt BM_GO/mfo_all_type2.txt > BM_GO_all_ont/type2.txt 
#cat BM_GO/bpo_all_type3.txt BM_GO/cco_all_type3.txt BM_GO/mfo_all_type3.txt > BM_GO_all_ont/type3.txt 
#cat BM_GO/bpo_all_type12.txt BM_GO/cco_all_type12.txt BM_GO/mfo_all_type12.txt > BM_GO_all_ont/type12.txt 

In [ ]:
/data/rashika/CAFA4/head2head/eval/

#Concat all three ontologies of each evaluation type together

# Commands run in /data/rashika/CAFA4/head2head/eval directory

cat bm_GO_cafa2/bpo_all_type1.txt bm_GO_cafa2/cco_all_type1.txt bm_GO_cafa2/mfo_all_type1.txt > BM_GO_all_ont/cafa2/type1.txt 
cat bm_GO_cafa2/bpo_all_type2.txt bm_GO_cafa2/cco_all_type2.txt bm_GO_cafa2/mfo_all_type2.txt > BM_GO_all_ont/cafa2/type2.txt 
cat bm_GO_cafa2/bpo_all_type3.txt bm_GO_cafa2/cco_all_type3.txt bm_GO_cafa2/mfo_all_type3.txt > BM_GO_all_ont/cafa2/type3.txt 
cat bm_GO_cafa2/bpo_all_type12.txt bm_GO_cafa2/cco_all_type12.txt bm_GO_cafa2/mfo_all_type12.txt > BM_GO_all_ont/cafa2/type12.txt 

cat bm_GO_cafa3/bpo_all_type1.txt bm_GO_cafa3/cco_all_type1.txt bm_GO_cafa3/mfo_all_type1.txt > BM_GO_all_ont/cafa3/type1.txt 
cat bm_GO_cafa3/bpo_all_type2.txt bm_GO_cafa3/cco_all_type2.txt bm_GO_cafa3/mfo_all_type2.txt > BM_GO_all_ont/cafa3/type2.txt 
cat bm_GO_cafa3/bpo_all_type3.txt bm_GO_cafa3/cco_all_type3.txt bm_GO_cafa3/mfo_all_type3.txt > BM_GO_all_ont/cafa3/type3.txt 
cat bm_GO_cafa3/bpo_all_type12.txt bm_GO_cafa3/cco_all_type12.txt bm_GO_cafa3/mfo_all_type12.txt > BM_GO_all_ont/cafa3/type12.txt 

cat bm_GO_cafa4/bpo_all_type1.txt bm_GO_cafa4/cco_all_type1.txt bm_GO_cafa4/mfo_all_type1.txt > BM_GO_all_ont/cafa4/type1.txt 
cat bm_GO_cafa4/bpo_all_type2.txt bm_GO_cafa4/cco_all_type2.txt bm_GO_cafa4/mfo_all_type2.txt > BM_GO_all_ont/cafa4/type2.txt 
cat bm_GO_cafa4/bpo_all_type3.txt bm_GO_cafa4/cco_all_type3.txt bm_GO_cafa4/mfo_all_type3.txt > BM_GO_all_ont/cafa4/type3.txt 
cat bm_GO_cafa4/bpo_all_type12.txt bm_GO_cafa4/cco_all_type12.txt bm_GO_cafa4/mfo_all_type12.txt > BM_GO_all_ont/cafa4/type12.txt 

In [ ]:
# roots is a list of the root terms
def create_bm_lists_h2h(t0_file, t1_file, t0_ont_graph, t1_ont_graph, t1_minus_ont_graph,roots, c2_t_minus_1_ont_graph, c3_t_minus_1_ont_graph, BM_path = "/data/rashika/CAFA4/head2head/eval/BM_GO/", remove_protein_binding = True):

    #Prop t0 and t1 in their respective ontologies
    t0_prop = process_raw_annot(t0_file, t0_ont_graph, roots)
    t1_prop = process_raw_annot(t1_file, t1_ont_graph, roots)
    
    # Keep terms common in t0_ont and t1_ont
    t0_common, t1_common =  keep_common_go_terms(t0_prop, t1_prop, t0_ont_graph, t1_ont_graph)
    print(len(t0_common), len(t1_common))
    
    # Keep terms common in (t0_ont, t1_ont) and (c2_t_minus_1,, c3_tminus_1) 
    t0_common, t1_common =  keep_common_go_terms(t0_common, t1_common, c2_t_minus_1_ont_graph, c2_t_minus_1_ont_graph)
    print(len(t0_common), len(t1_common))

    # Propagate back in the t(-1) ontology
    subontologies = {aspect: fetch_aspect(t1_minus_ont_graph, roots[aspect]) for aspect in roots}
    t0_eval = propagate_terms(t0_common, subontologies)
    t1_eval = propagate_terms(t1_common, subontologies)

    # Convert the eval Dfs into annotation lists
    t0_annot_list = get_annot_list(t0_eval)
    t1_annot_list = get_annot_list(t1_eval)
    
    # get baselines
    get_baselines(t0_annot_list, t1_annot_list,  BM_path, remove_protein_binding = True)

In [ ]:
roots = {'BPO': 'GO:0008150', 'CCO': 'GO:0005575', 'MFO': 'GO:0003674'}
#create_bm_lists_h2h(t0_mapped_path,t1_mapped_path, t0_ont_graph, t1_ont_graph, t_minus_1_ont_graph,roots, c2_t_minus_1_ont_graph, c3_t_minus_1_ont_graph, BM_path = "/data/rashika/CAFA4/head2head/eval/BM_GO/")

### Use the created benchmarks to map to the benchmarks that are in all the CAFAs

In [6]:
h2h_proteins = pd.read_csv('/data/common/CAFA4/h2h/H2H_CAFA4.tsv', sep = '\t', header = 0)
display(h2h_proteins)
BM_path = "/data/rashika/CAFA4/head2head/eval/BM_GO/"

,AC ID,Sequence,Taxon ID,CAFA_ID_2,CAFA_ID_3,CAFA_ID_4,Entry_ID_2,Entry_ID_3,Entry_ID_4
0,Q9CQV8,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...,10090,T100900000001,T100900000001,T100900000001,1433B_MOUSE,1433B_MOUSE,1433B_MOUSE
1,P62259,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...,10090,T100900000002,T100900000002,T100900000002,1433E_MOUSE,1433E_MOUSE,1433E_MOUSE
2,P68510,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...,10090,T100900000003,T100900000003,T100900000003,1433F_MOUSE,1433F_MOUSE,1433F_MOUSE
3,P61982,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...,10090,T100900000004,T100900000004,T100900000004,1433G_MOUSE,1433G_MOUSE,1433G_MOUSE
4,O70456,MERASLIQKAKLAEQAERYEDMAAFMKSAVEKGEELSCEERNLLSV...,10090,T100900000005,T100900000005,T100900000005,1433S_MOUSE,1433S_MOUSE,1433S_MOUSE
...,...,...,...,...,...,...,...,...,...
76514,Q8ZNV7,MTSLVSLENVSVSFGQRRVLSDVSLELSPGKILTLLGPNGAGKSTL...,99287,T992870001767,T992870001785,T992870001793,ZNUC_SALTY,ZNUC_SALTY,ZNUC_SALTY
76515,Q9L9I0,MKRNNKSAIALIALSLLALSSGAAFAGHHWGNNDGMWQQGGSPLTT...,99287,T992870001768,T992870001786,T992870001794,ZRAP_SALTY,ZRAP_SALTY,ZRAP_SALTY
76516,P25852,MIRGKIDILVVDDDVSHCTILQALLRGWGYNVALAYSGHDALAQVR...,99287,T992870001769,T992870001787,T992870001795,ZRAR_SALTY,ZRAR_SALTY,ZRAR_SALTY
76517,P37461,MSFIRLHKDAAATWLSRLLPAAIFILVGLFSIMVIRDYGRESAAAR...,99287,T992870001770,T992870001788,T992870001796,ZRAS_SALTY,ZRAS_SALTY,ZRAS_SALTY


In [ ]:
BM_list = os.listdir(BM_path)
mapping_4_to_3 = dict(zip(h2h_proteins['CAFA_ID_4'], h2h_proteins['CAFA_ID_3']))
mapping_4_to_2 = dict(zip(h2h_proteins['CAFA_ID_4'], h2h_proteins['CAFA_ID_2']))
mapping_4_to_4 = dict(zip(h2h_proteins['CAFA_ID_4'], h2h_proteins['CAFA_ID_4']))
for bm_file in BM_list:
    print(bm_file)
    bm_df = pd.read_csv(BM_path+bm_file, header = None, sep = '\t')
    bm_df.columns = ['ID', 'GO ID', 'aspect']
    
    # Shortlist the proteins to the h2h common proteins
    bm_df = bm_df[bm_df['ID'].isin(h2h_proteins['CAFA_ID_4'])].copy()
    
    # Map to IDs from respective CAFAs
    bm_df['CAFA3_ID'] = bm_df['ID'].map(mapping_4_to_3)
    bm_df['CAFA2_ID'] = bm_df['ID'].map(mapping_4_to_2)
    bm_df['CAFA4_ID'] = bm_df['ID'].map(mapping_4_to_4)
    
    # Get the benchmark copy for respective CAFA
    bm_df_cafa4 = bm_df.loc[:, ["CAFA4_ID", "GO ID", "aspect"]].copy()
    bm_df_cafa3 = bm_df.loc[:, ["CAFA3_ID", "GO ID", "aspect"]].copy()
    bm_df_cafa2 = bm_df.loc[:, ["CAFA2_ID", "GO ID", "aspect"]].copy()
    
    # Make the directory to save the mapped benchmarks for each cafa
    #os.mkdir('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa4')
    #os.mkdir('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa3')
    #os.mkdir('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa2')
    bm_df_cafa4.to_csv('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa4/' + bm_file, sep = '\t', header = None, index = False)
    bm_df_cafa3.to_csv('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa3/' + bm_file, sep = '\t', header = None, index = False)
    bm_df_cafa2.to_csv('/data/rashika/CAFA4/head2head/eval/bm_GO_cafa2/' + bm_file, sep = '\t', header = None, index = False)
    
    display(bm_df)

In [ ]:
# Test a protein mapping
h2h_proteins.loc[h2h_proteins["CAFA_ID_4"] == "T100900005994"]

### Run the Evaluation for CAFA4 H2H

In [ ]:
# BM_GO_path = "/data/rashika/CAFA4/head2head/eval/bm_GO_cafa4/"
# pred_dir = "/data/yisupeng/sharing/cafa4/all_models/"
# t_minus_ont_file = "/data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo"
# IA_file = "/data/rashika/CAFA4/eval_final/IA.txt"
# result_path = "/data/rashika/CAFA4/head2head/eval/cafa4_eval/"
# #run_eval(BM_GO_path, pred_dir, t_minus_ont_file, IA_file, result_path, log_path = '/data/rashika/CAFA4/head2head/eval/cafa4_log/', thresh_step = 0.01)

In [5]:
BM_GO_path = "/data/rashika/CAFA4/head2head/eval/bm_GO_cafa4/"
pred_dir = "/data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/"
t_minus_ont_file = "/data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo"
IA_file = "/data/rashika/CAFA4/eval_final/IA.txt"
result_path = "/data/rashika/CAFA4/head2head/eval/cafa4_part2_eval/"
log_path = "/data/rashika/CAFA4/head2head/eval/cafa4_part2_log/"
run_eval(BM_GO_path, pred_dir, t_minus_ont_file, IA_file, result_path, log_path = log_path, thresh_step = 0.01)

Evaluating: bpo_all_type1.txt
python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py /data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/ /data/rashika/CAFA4/head2head/eval/bm_GO_cafa4/bpo_all_type1.txt -out_dir /data/rashika/CAFA4/head2head/eval/cafa4_part2_eval/bpo_all_type1/ -ia /data/rashika/CAFA4/eval_final/IA.txt -prop fill -threads 1 -th_step 0.01 -no_orphans > /data/rashika/CAFA4/head2head/eval/cafa4_part2_log/bpo_all_type1/run.log &
Evaluating: cco_all_type1.txt
python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py /data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/ /data/rashika/CAFA4/head2head/eval/bm_GO_cafa4/cco_all_type1.txt -out_dir /data/rashika/CAFA4/head2head/eval/cafa4_part2_eval/cco_all_type1/ -ia /data/rashika/CAFA4/eval_final/IA.txt -prop fill -threads 1 -th_step 0.01 -no_orphans > /data/rashika/CAFA4/head2head/eval/cafa

2024-07-09 16:00:08,048 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-09 16:00:08,087 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-09 16:00:08,120 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-09 16:00:08,171 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-09 16:00:08,178 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-09 16:00:08,240 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-09 16:00:08,273 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-09 16:00:08,276 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-09 16:00:08,292 [INFO ] Ontology: biolog

2024-07-09 16:00:49,367 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M101, evaluated
2024-07-09 16:00:49,560 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M101, evaluated
2024-07-09 16:00:50,373 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M100, evaluated
2024-07-09 16:00:50,578 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M102, cellular_component, proteins 9, annotations 619, replaced alt. ids 0
2024-07-09 16:00:50,793 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M102, cellular_component, proteins 24, annotations 1301, replaced alt. ids 0
2024-07-09 16:00:55,392 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M101, cellular_component, proteins 1079, annotations 23292, replaced alt. ids 0
2024-07-09 16:01:07,499 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M102, evaluated
2024

2024-07-09 16:04:00,887 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M105, evaluated
2024-07-09 16:04:05,752 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M109, evaluated
2024-07-09 16:04:07,860 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M101, evaluated
2024-07-09 16:04:12,914 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M102, biological_process, proteins 25, annotations 7987, replaced alt. ids 0
2024-07-09 16:04:22,855 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M110, cellular_component, proteins 930, annotations 85345, replaced alt. ids 0
2024-07-09 16:04:26,782 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M015, evaluated
2024-07-09 16:04:30,186 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M110, cellular_component, proteins 945, annotations 84816, replaced alt. ids 0
2024-07-09 16:0

2024-07-09 16:08:43,289 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M114, evaluated
2024-07-09 16:08:45,535 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M104, evaluated
2024-07-09 16:08:46,293 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M105, molecular_function, proteins 1286, annotations 40325, replaced alt. ids 0
2024-07-09 16:08:47,144 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M115, cellular_component, proteins 908, annotations 34496, replaced alt. ids 0
2024-07-09 16:08:47,515 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M110, evaluated
2024-07-09 16:08:49,893 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M115, cellular_component, proteins 945, annotations 36072, replaced alt. ids 0
2024-07-09 16:08:52,655 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M111, cellular_component, pr

...

2024-07-09 16:57:59,989 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M130, evaluated
2024-07-09 16:58:00,726 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M108, biological_process, proteins 18891, annotations 3148438, replaced alt. ids 65103
2024-07-09 16:58:03,638 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M131, molecular_function, proteins 1582, annotations 11225, replaced alt. ids 0
2024-07-09 16:58:10,825 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M021, molecular_function, proteins 860, annotations 60519, replaced alt. ids 323
2024-07-09 16:58:24,874 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M121, evaluated
2024-07-09 16:58:27,670 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M089, cellular_component, proteins 1875, annotations 144916, replaced alt. ids 0
2024-07-09 16:58:34,507 [INFO ] Prediction: /data/rashika/CAFA4/hea

2024-07-09 17:07:55,118 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M114, evaluated
2024-07-09 17:08:04,647 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M115, molecular_function, proteins 3840, annotations 70859, replaced alt. ids 0
2024-07-09 17:08:06,887 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M130, evaluated
2024-07-09 17:08:20,930 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M131, biological_process, proteins 963, annotations 35167, replaced alt. ids 0
2024-07-09 17:08:37,672 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M125, evaluated
2024-07-09 17:08:44,435 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M126, molecular_function, proteins 1130, annotations 17531, replaced alt. ids 0
2024-07-09 17:08:59,218 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M136, evaluat

2024-07-09 17:18:07,040 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M114, cellular_component, proteins 11729, annotations 5060793, replaced alt. ids 2507
2024-07-09 17:18:20,061 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M129, evaluated
2024-07-09 17:18:30,259 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M130, molecular_function, proteins 2002, annotations 38635, replaced alt. ids 0
2024-07-09 17:18:49,273 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M113, evaluated
2024-07-09 17:19:13,213 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M142, evaluated
2024-07-09 17:19:20,481 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M143, molecular_function, proteins 1641, annotations 155278, replaced alt. ids 0
2024-07-09 17:19:28,351 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M135,

2024-07-09 17:27:24,729 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M134, molecular_function, proteins 11, annotations 88, replaced alt. ids 0
2024-07-09 17:27:30,875 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M116, cellular_component, proteins 2093, annotations 49403, replaced alt. ids 0
2024-07-09 17:27:32,838 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M093, molecular_function, proteins 1607, annotations 151710, replaced alt. ids 0
2024-07-09 17:27:39,331 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M109, evaluated
2024-07-09 17:28:38,452 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M140, evaluated
2024-07-09 17:28:42,856 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M141, biological_process, proteins 1018, annotations 4892, replaced alt. ids 0
2024-07-09 17:28:49,487 [INFO ] Prediction: /data/rashika/CAFA4/hea

2024-07-09 17:37:50,495 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M136, biological_process, proteins 727, annotations 21940, replaced alt. ids 0
2024-07-09 17:37:55,559 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M121, evaluated
2024-07-09 17:38:06,149 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M122, molecular_function, proteins 720, annotations 42747, replaced alt. ids 0
2024-07-09 17:38:10,773 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M092, biological_process, proteins 1018, annotations 118689, replaced alt. ids 0
2024-07-09 17:38:15,485 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M090, evaluated
2024-07-09 17:38:21,599 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M117, evaluated
2024-07-09 17:38:24,622 [INFO ] Prediction: /data/rashika/CAFA4/head2head/CAFA4/cafa4_submissions_2/M098, evaluated
2024

### Run the Evaluation for CAFA3 H2H

In [2]:
BM_GO_path = "/data/rashika/CAFA4/head2head/eval/bm_GO_cafa3/"
pred_dir = "/data/yisupeng/sharing/cafa3/consolidated/PASSED/"
t_minus_ont_file = "/data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo"
IA_file = "/data/rashika/CAFA4/eval_final/IA.txt"
result_path = "/data/rashika/CAFA4/head2head/eval/cafa3_eval/"
log_path = '/data/rashika/CAFA4/head2head/eval/cafa3_log/'
run_eval(BM_GO_path, pred_dir, t_minus_ont_file, IA_file, result_path, log_path = log_path, thresh_step = 0.01)

Evaluating: bpo_all_type1.txt
python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py /data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo /data/yisupeng/sharing/cafa3/consolidated/PASSED/ /data/rashika/CAFA4/head2head/eval/bm_GO_cafa3/bpo_all_type1.txt -out_dir /data/rashika/CAFA4/head2head/eval/cafa3_eval/bpo_all_type1/ -ia /data/rashika/CAFA4/eval_final/IA.txt -prop fill -threads 1 -th_step 0.01 -no_orphans > /data/rashika/CAFA4/head2head/eval/cafa3_log/bpo_all_type1/run.log &
Evaluating: cco_all_type1.txt
python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py /data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo /data/yisupeng/sharing/cafa3/consolidated/PASSED/ /data/rashika/CAFA4/head2head/eval/bm_GO_cafa3/cco_all_type1.txt -out_dir /data/rashika/CAFA4/head2head/eval/cafa3_eval/cco_all_type1/ -ia /data/rashika/CAFA4/eval_final/IA.txt -prop fill -threads 1 -th_step 0.01 -no_orphans > /data/rashika/CAFA4/head2head/eval/cafa3_log/cco_all_type1/run.log &
Ev

2024-07-08 18:16:46,506 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:16:46,542 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:16:46,542 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:16:46,584 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:16:46,606 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:16:46,624 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:16:46,685 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:16:46,769 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:16:46,833 [INFO ] Ontology: biolog

2024-07-08 18:17:19,465 [INFO ] Ground truth: biological_process, proteins 19376, annotations 469274, replaced alt. ids 0
2024-07-08 18:17:23,038 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M085, evaluated
2024-07-08 18:17:23,583 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M085, evaluated
2024-07-08 18:17:34,116 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M083, evaluated
2024-07-08 18:17:34,427 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:17:34,439 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M001, not evaluated
2024-07-08 18:17:35,411 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:17:35,416 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M002, not evaluated
2024-07-08 18:17:35,918 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:17:35,921 [WARNI] Prediction: /data

2024-07-08 18:21:02,098 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M105, cellular_component, proteins 907, annotations 80118, replaced alt. ids 1040
2024-07-08 18:21:05,428 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M085, evaluated
2024-07-08 18:21:14,711 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M105, evaluated
2024-07-08 18:21:14,766 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M105, evaluated
2024-07-08 18:21:15,233 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M083, evaluated
2024-07-08 18:21:15,492 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:21:15,495 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M001, not evaluated
2024-07-08 18:21:16,379 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M006, cellular_component, proteins 923, annotations 14008, replaced alt. ids 70
2024-07-08 

2024-07-08 18:23:51,460 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M009, evaluated
2024-07-08 18:23:53,929 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:23:53,949 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M110, not evaluated
2024-07-08 18:23:56,887 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:23:56,894 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M110, not evaluated
2024-07-08 18:23:58,132 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M010, cellular_component, proteins 916, annotations 32912, replaced alt. ids 169
2024-07-08 18:24:01,054 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M010, cellular_component, proteins 923, annotations 35422, replaced alt. ids 197
2024-07-08 18:24:05,071 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M105, evaluated
2024-07-08 18:24:07,074 

...

2024-07-08 18:40:22,388 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M033, cellular_component, proteins 930, annotations 122545, replaced alt. ids 1770
2024-07-08 18:40:23,646 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M010, molecular_function, proteins 1563, annotations 61888, replaced alt. ids 798
2024-07-08 18:40:28,801 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M108, biological_process, proteins 165, annotations 128370, replaced alt. ids 3487
2024-07-08 18:40:30,190 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M031, evaluated
2024-07-08 18:40:30,204 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:40:30,204 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M032, not evaluated
2024-07-08 18:40:38,038 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M033, evaluated
2024-07-08 18:40:44,489 [INFO ] Prediction: /data/

2024-07-08 18:43:01,324 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M041, evaluated
2024-07-08 18:43:01,348 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:43:01,348 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M042, not evaluated
2024-07-08 18:43:02,389 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:43:02,390 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M043, not evaluated
2024-07-08 18:43:04,489 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M044, cellular_component, proteins 56, annotations 646, replaced alt. ids 16
2024-07-08 18:43:07,525 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M040, evaluated
2024-07-08 18:43:08,102 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M041, cellular_component, proteins 39, annotations 460, replaced alt. ids 8
2024-07-08 18:43:19,919 [INFO ] P

2024-07-08 18:46:00,199 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M052, evaluated
2024-07-08 18:46:02,769 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M052, cellular_component, proteins 837, annotations 21897, replaced alt. ids 530
2024-07-08 18:46:03,922 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:46:03,926 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M110, not evaluated
2024-07-08 18:46:07,011 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M108, biological_process, proteins 333, annotations 275831, replaced alt. ids 7330
2024-07-08 18:46:09,527 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M010, molecular_function, proteins 2150, annotations 80014, replaced alt. ids 959
2024-07-08 18:46:13,910 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:46:13,912 [WARNI] Prediction: /data/yisupeng/sharing/ca

2024-07-08 18:49:45,363 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M059, evaluated
2024-07-08 18:49:54,151 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M124, cellular_component, proteins 1834, annotations 112496, replaced alt. ids 3398
2024-07-08 18:49:56,315 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M110, evaluated
2024-07-08 18:50:00,819 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M059, evaluated
2024-07-08 18:50:09,472 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M084, biological_process, proteins 18015, annotations 2543268, replaced alt. ids 72544
2024-07-08 18:50:10,080 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M010, biological_process, proteins 980, annotations 185246, replaced alt. ids 433
2024-07-08 18:50:13,124 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M005, evaluated
2024-07-08 18:50:18,711 [INFO ] 

2024-07-08 18:53:34,719 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M065, evaluated
2024-07-08 18:53:36,584 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M066, cellular_component, proteins 930, annotations 60514, replaced alt. ids 1196
2024-07-08 18:53:37,037 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M105, evaluated
2024-07-08 18:53:45,778 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M028, evaluated
2024-07-08 18:53:45,782 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:53:45,797 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M029, not evaluated
2024-07-08 18:53:45,799 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 18:53:45,800 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M030, not evaluated
2024-07-08 18:53:50,603 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PA

### Run for CAFA2 H2H

In [3]:
BM_GO_path = "/data/rashika/CAFA4/head2head/eval/bm_GO_cafa2/"
pred_dir = "/data/yisupeng/sharing/cafa2/consolidated/"
t_minus_ont_file = "/data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo"
IA_file = "/data/rashika/CAFA4/eval_final/IA.txt"
result_path = "/data/rashika/CAFA4/head2head/eval/cafa2_eval/"
log_path = '/data/rashika/CAFA4/head2head/eval/cafa2_log/'
run_eval(BM_GO_path, pred_dir, t_minus_ont_file, IA_file, result_path, log_path = log_path, thresh_step = 0.01)

Evaluating: bpo_all_type1.txt
python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py /data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo /data/yisupeng/sharing/cafa2/consolidated/ /data/rashika/CAFA4/head2head/eval/bm_GO_cafa2/bpo_all_type1.txt -out_dir /data/rashika/CAFA4/head2head/eval/cafa2_eval/bpo_all_type1/ -ia /data/rashika/CAFA4/eval_final/IA.txt -prop fill -threads 1 -th_step 0.01 -no_orphans > /data/rashika/CAFA4/head2head/eval/cafa2_log/bpo_all_type1/run.log &
Evaluating: cco_all_type1.txt
python3 /home/rashika/CAFA4/CAFA-evaluator/src/cafaeval/__main__.py /data/rashika/CAFA4/uniprot/go_2019_10_07/go-basic.obo /data/yisupeng/sharing/cafa2/consolidated/ /data/rashika/CAFA4/head2head/eval/bm_GO_cafa2/cco_all_type1.txt -out_dir /data/rashika/CAFA4/head2head/eval/cafa2_eval/cco_all_type1/ -ia /data/rashika/CAFA4/eval_final/IA.txt -prop fill -threads 1 -th_step 0.01 -no_orphans > /data/rashika/CAFA4/head2head/eval/cafa2_log/cco_all_type1/run.log &
Evaluating: mfo_

2024-07-08 18:56:14,220 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:56:14,231 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:56:14,251 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:56:14,257 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:56:14,279 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:56:14,294 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:56:14,331 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:56:14,338 [INFO ] Ontology: biological_process, total 29456, roots 1, leaves 13559, alternative_ids 1474
2024-07-08 18:56:14,357 [INFO ] Ontology: biolog

2024-07-08 18:56:24,982 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M006, cellular_component, proteins 12982, annotations 339239, replaced alt. ids 2034
2024-07-08 18:56:26,241 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M034, evaluated
2024-07-08 18:56:28,335 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M035, cellular_component, proteins 1635, annotations 39452, replaced alt. ids 40
2024-07-08 18:56:28,661 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M015, molecular_function, proteins 1484, annotations 44534, replaced alt. ids 796
2024-07-08 18:56:31,092 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M071, evaluated
2024-07-08 18:56:36,735 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M071, cellular_component, proteins 945, annotations 51107, replaced alt. ids 1970
2024-07-08 18:56:38,778 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consoli

2024-07-08 18:58:30,895 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M010, biological_process, proteins 1208, annotations 208769, replaced alt. ids 570
2024-07-08 18:58:35,070 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M074, evaluated
2024-07-08 18:58:35,704 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M107, biological_process, proteins 2253, annotations 1509086, replaced alt. ids 38070
2024-07-08 18:58:35,994 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M073, evaluated
2024-07-08 18:58:43,119 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M075, cellular_component, proteins 378, annotations 13106, replaced alt. ids 145
2024-07-08 18:58:43,993 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M074, cellular_component, proteins 495, annotations 16597, replaced alt. ids 231
2024-07-08 18:58:51,532 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consol

2024-07-08 19:00:35,893 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M027, evaluated
2024-07-08 19:00:39,752 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M025, evaluated
2024-07-08 19:00:44,960 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M078, evaluated
2024-07-08 19:00:49,070 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M012, evaluated
2024-07-08 19:00:50,550 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M079, evaluated
2024-07-08 19:00:51,246 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M010, evaluated
2024-07-08 19:00:55,663 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M030, cellular_component, proteins 921, annotations 101031, replaced alt. ids 2845
2024-07-08 19:00:58,416 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M028, cellular_component, proteins 1843, annotations 117565, replaced alt. ids 3565
2024-07-08 19:00:58,5

...

2024-07-08 20:02:09,949 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M045, biological_process, proteins 1246, annotations 37177, replaced alt. ids 26
2024-07-08 20:02:13,865 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M014, cellular_component, proteins 11792, annotations 3452620, replaced alt. ids 23467
2024-07-08 20:02:21,226 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M097, evaluated
2024-07-08 20:02:25,901 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M098, cellular_component, proteins 1539, annotations 16985, replaced alt. ids 205
2024-07-08 20:02:46,029 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M054, evaluated
2024-07-08 20:02:56,230 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M055, molecular_function, proteins 1553, annotations 109493, replaced alt. ids 2408
2024-07-08 20:02:59,115 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M098, evaluated
2024-07-0

2024-07-08 20:07:13,970 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M113, evaluated
2024-07-08 20:07:13,980 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 20:07:14,048 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M013, not evaluated
2024-07-08 20:07:18,015 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M041, evaluated
2024-07-08 20:07:21,630 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M042, biological_process, proteins 1, annotations 26, replaced alt. ids 0
2024-07-08 20:07:30,211 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M103, evaluated
2024-07-08 20:07:34,819 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M104, cellular_component, proteins 1846, annotations 24588, replaced alt. ids 454
2024-07-08 20:07:37,803 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M114, biological_process, proteins 1801, annota

2024-07-08 20:11:54,447 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M059, evaluated
2024-07-08 20:12:04,666 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M036, evaluated
2024-07-08 20:12:10,891 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M048, evaluated
2024-07-08 20:12:13,499 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 20:12:13,500 [WARNI] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M111, not evaluated
2024-07-08 20:12:15,533 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 20:12:15,534 [WARNI] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M112, not evaluated
2024-07-08 20:12:17,590 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 20:12:17,590 [WARNI] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M113, not evaluated
2024-07-08 20:12:18,682 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/

2024-07-08 20:16:18,786 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M050, biological_process, proteins 1245, annotations 1596067, replaced alt. ids 34710
2024-07-08 20:16:27,025 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M061, evaluated
2024-07-08 20:16:29,462 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 20:16:29,463 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M031, not evaluated
2024-07-08 20:16:29,470 [WARNI] Empty prediction! Check format or overlap with ground truth
2024-07-08 20:16:29,471 [WARNI] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M032, not evaluated
2024-07-08 20:16:31,708 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M055, evaluated
2024-07-08 20:16:31,940 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M119, evaluated
2024-07-08 20:16:33,940 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M063, molecular_fun

2024-07-08 20:20:08,662 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M063, evaluated
2024-07-08 20:20:10,675 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M067, evaluated
2024-07-08 20:20:12,952 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M068, molecular_function, proteins 993, annotations 11531, replaced alt. ids 48
2024-07-08 20:20:15,286 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M051, biological_process, proteins 1245, annotations 1399418, replaced alt. ids 34513
2024-07-08 20:20:16,896 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M057, evaluated
2024-07-08 20:20:23,255 [INFO ] Prediction: /data/yisupeng/sharing/cafa3/consolidated/PASSED/M064, molecular_function, proteins 1629, annotations 83560, replaced alt. ids 3278
2024-07-08 20:20:29,700 [INFO ] Prediction: /data/yisupeng/sharing/cafa2/consolidated/M125, evaluated
2024-07-08 20:20:34,554 [WARNI] Empty prediction! Check format or overl